# Ukrainian refugees entering Poland

*March 22, 2022*

Yesterday, I did some analysis on UNHCR refugee data and how many refugees are entering countries neighbouring Ukraine. Today, I want to focus in on Poland for a few graphics. First, import pandas and requests, then read in raw data from previous years from UNHCR.

In [14]:
import pandas as pd
import requests

raw = pd.read_csv('../raw/RAW 2022 UNHCR REFUGEES.csv', skiprows=14)

display(raw.head())

,Year,Country of origin,Country of origin (ISO),Country of asylum,Country of asylum (ISO),Refugees under UNHCR's mandate,Asylum-seekers,IDPs of concern to UNHCR,Venezuelans displaced abroad,Stateless persons,Others of concern
0,2016,Afghanistan,AFG,Afghanistan,AFG,0,0,1797551,NaN,0,114221
1,2016,Afghanistan,AFG,Egypt,EGY,14,21,0,NaN,0,0
2,2016,Afghanistan,AFG,Argentina,ARG,5,0,0,NaN,0,0
3,2016,Afghanistan,AFG,Australia,AUS,9217,2077,0,NaN,0,0
4,2016,Afghanistan,AFG,Austria,AUT,20220,29783,0,NaN,0,0


### Poland's crisis refugees vs. crisis refugees in other neighbouring countries

Next, we'll grab the Ukrainian crisis refugee data via this API.

In [23]:
r = (requests
     .get('https://data2.unhcr.org/population/get/sublocation?widget_id=286723&sv_id=54&population_group=5461&forcesublocation=0&fromDate=1900-01-01')
     .json()
     ["data"]
     )

crisis_refugees = (pd
        .DataFrame(r)
        .loc[:, ["geomaster_name", "individuals"]]
        .set_index("geomaster_name")
        )

display(crisis_refugees)

,individuals
geomaster_name,
Poland,2113554
Romania,543308
Republic of Moldova,367913
Hungary,317863
Slovakia,253592
Russian Federation,252376
Belarus,4308


I used this data to create [this visualization](https://www.datawrapper.de/_/PwuYO/).

### Poland's crisis refugees vs. previous years

Now let's pick out Poland from the raw data so we can compare these years to the crisis data.

In [16]:
poland = (raw
            .loc[raw["Country of asylum"] == "Poland", ["Country of asylum", "Year", "Refugees under UNHCR's mandate"]]
            .pivot_table(columns="Year", index="Country of asylum", aggfunc='sum')
            .droplevel(0, axis=1)
            .sort_values(2021, ascending=False)
            )

display(poland)

Year,2016,2017,2018,2019,2020,2021
Country of asylum,,,,,,
Poland,11738,12225,12495,12658,2771,3166


Now, we sum 2016-2021, then tack on our crisis data for Poland to create a table with all the information we need for our next graphic.

In [21]:
poland["sum"] = poland.loc[:,2016:2021].sum(axis=1)

table = (poland.join(crisis_refugees)
         .rename(columns={"individuals": "crisis"})
         .astype(int)
         .transpose()
         .rename(columns={"Poland": "Refugees"})
         )

display(table)

Country of asylum,Refugees
2016,11738
2017,12225
2018,12495
2019,12658
2020,2771
2021,3166
sum,55053
crisis,2113554


This table will be used to create [this graphic](https://www.datawrapper.de/_/XHWMo/), showing just how massive the influx of refugees to Poland is.

\-30\-